In [1]:
from datetime import datetime
from nba import NbaTracker
import importlib
import nba
importlib.reload(nba)

# data starts from 2000-01
seasons = [
    "2000-01",
    "2001-02",
    "2002-03",
    "2003-04",
    "2004-05",
    "2005-06",
    "2006-07",
    "2007-08",
    "2008-09",
    "2009-10",
    "2010-11",
    "2011-12",
    "2012-13",
    "2013-14",
    "2014-15",
    "2015-16",
    "2016-17",
    "2017-18",
    "2018-19",
    "2019-20",
    "2020-21"
]  # 19-20 was bubble season

nbaTracker = NbaTracker()
for season_name in seasons[:5]:
    nbaTracker.load_season(season_name)
    print("Loaded ", season_name, datetime.now().time())

Loaded  2000-01 16:19:20.074786
Loaded  2001-02 16:19:29.311277
Loaded  2002-03 16:19:37.126806
Loaded  2003-04 16:19:43.592080
Loaded  2004-05 16:19:51.426698


In [2]:
import pandas as pd
df = pd.read_csv("C:/Users/bhalb/nbaproject/data/possession_data.csv")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, random_state=5325)


In [ ]:
plot = train.sample(n=100000).copy()


In [ ]:
plot.plot(kind='scatter', x="time", y="score_margin", alpha=0.1, c=plot["is_win"], cmap=plt.get_cmap("jet"), colorbar=True, figsize=(
    10, 7))

In [ ]:
plot["time"] = (2880 - plot["seconds_left"])


In [ ]:
plot["timescore"] = plot["time"] * plot["score_margin"]
plot["timescore"].hist()


In [ ]:
plot["certainty"] = np.abs(plot["score_margin"]) * plot["time"]


In [ ]:
X_train = train.drop("is_win", axis=1)


In [ ]:
X_train.values[:,0].max()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
# transformers act on array not df
left_ix = 0
score_ix = 1
home_ix = 2


class CombinedAttrAdder(BaseEstimator, TransformerMixin):
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        self.max_time = X[:, left_ix].max()
        return self

    def transform(self, X, y=None):
        time = (self.max_time - X[:, left_ix]) / self.max_time
        timescore = time * X[:, score_ix]
        return np.c_[X[:, home_ix], time, X[:, score_ix], timescore]


In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

ct = make_column_transformer(
    ('passthrough', [0, 1]), (StandardScaler(), [2, 3]),)
pipeline = Pipeline([('attr_added', CombinedAttrAdder()), ('ct', ct)])


In [ ]:
X_train = pipeline.fit_transform(train.drop("is_win", axis=1).to_numpy())
y_train = train["is_win"]


In [ ]:
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train, y_train.ravel())


In [ ]:
test = np.array([[200, -2, 0]])
clf.predict_proba(pipeline.transform(test))


In [ ]:
print(clf.predict(X_train[:10]), '\n', y_train.values[:10])
